In [275]:
import pandas as pd
import xmltodict
import requests
url = "http://ergast.com/api/f1/drivers.json?callback=myParser"
url1 = "http://ergast.com/api/f1/drivers.json"
url2 = "http://ergast.com/api/f1/2021"
response = requests.get(url2)

In [276]:
'''
Checking if the parsed data is in a json file
Trying to parse an empty response as if it were JSON would raise a JSONDecodeError, so we have to check if the response status is not 204, 
the response headers dict has a content-type key and the value of the `content-type` header is application/json.
'''
response = requests.delete(url2)

print('response: 👉️', response)  # response: 👉️ <Response [204]>
print('response.text: 👉️', response.text)  # response.text: 👉️ ""

# response.status_code: 👉️ 204
print('response.status_code: 👉️', response.status_code)
print('response.headers: 👉️', response.headers)

if (response.status_code != 204 and 'content-type' in response.headers and 'application/json' in response.headers['content-type']):
    parsed = response.json()
    print('✅ parsed response: 👉️', parsed)
else:
# 👇️ this runs
    print('⛔️ conditions not met')

response: 👉️ <Response [200]>
response.text: 👉️ <?xml version="1.0" encoding="utf-8"?>
<?xml-stylesheet type="text/xsl" href="/schemas/mrd-1.5.xsl"?>
<MRData xmlns="http://ergast.com/mrd/1.5" series="f1" url="http://ergast.com/api/f1/2021" limit="30" offset="0" total="22">
	<RaceTable season="2021">
		<Race season="2021" round="1" url="http://en.wikipedia.org/wiki/2021_Bahrain_Grand_Prix">
			<RaceName>Bahrain Grand Prix</RaceName>
			<Circuit circuitId="bahrain" url="http://en.wikipedia.org/wiki/Bahrain_International_Circuit">
				<CircuitName>Bahrain International Circuit</CircuitName>
				<Location lat="26.0325" long="50.5106">
					<Locality>Sakhir</Locality>
					<Country>Bahrain</Country>
				</Location>
			</Circuit>
			<Date>2021-03-28</Date>
			<Time>15:00:00Z</Time>
			<FirstPractice>
				<Date>2021-03-26</Date>
			</FirstPractice>
			<SecondPractice>
				<Date>2021-03-26</Date>
			</SecondPractice>
			<ThirdPractice>
				<Date>2021-03-27</Date>
			</ThirdPractice>
			<Qualif

In [277]:
#Exploring response attributes
print(response.url)
print(response.history)
print(response.status_code)
print(response.headers)
print(response.headers['content-type'])

http://ergast.com/api/f1/2021
[]
200
{'Date': 'Mon, 12 Sep 2022 06:18:57 GMT', 'Server': 'Apache/2.4.6 (CentOS) OpenSSL/1.0.2k-fips PHP/5.4.16', 'X-Powered-By': 'PHP/5.4.16', 'Access-Control-Allow-Origin': '*', 'Cache-Control': 'public, max-age=300', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/xml; charset=utf-8'}
application/xml; charset=utf-8


In [278]:
# Reading the XML files and converting them to data frames
races = pd.DataFrame.from_dict(xmltodict.parse(response.text)['MRData']['RaceTable']['Race'])

In [279]:
races.head(1)

,@season,@round,@url,RaceName,Circuit,Date,Time,FirstPractice,SecondPractice,ThirdPractice,Qualifying,Sprint
0,2021,1,http://en.wikipedia.org/wiki/2021_Bahrain_Gran...,Bahrain Grand Prix,"{'@circuitId': 'bahrain', '@url': 'http://en.w...",2021-03-28,15:00:00Z,{'Date': '2021-03-26'},{'Date': '2021-03-26'},{'Date': '2021-03-27'},{'Date': '2021-03-27'},NaN


In [280]:
# Treating races data frame

races['ThirdPractice'] = races['ThirdPractice'].apply(lambda x: x if x == x else {'Date': 'NA'})

def races_dates(i):
    races[i] = races[i].transform(lambda x: x['Date'])

[races_dates(i) for i in ['FirstPractice', 'SecondPractice', 'ThirdPractice', 'Qualifying']];

races['CircuitID'] = races['Circuit'].transform(lambda x: x['@circuitId'])
races['CircuitName'] = races['Circuit'].transform(lambda x: x['CircuitName'])
races['Locality'] = races['Circuit'].transform(lambda x: x['Location']['Locality'])
races['Country'] = races['Circuit'].transform(lambda x: x['Location']['Country'])

races.drop(columns=['@url', 'Circuit'], inplace=True)

races.rename(columns={"@season": "Season", "@round": "round", "Date": "Race"}, errors="raise", inplace=True)

In [281]:
races.head(1)

,Season,round,RaceName,Race,Time,FirstPractice,SecondPractice,ThirdPractice,Qualifying,Sprint,CircuitID,CircuitName,Locality,Country
0,2021,1,Bahrain Grand Prix,2021-03-28,15:00:00Z,2021-03-26,2021-03-26,2021-03-27,2021-03-27,NaN,bahrain,Bahrain International Circuit,Sakhir,Bahrain
